# Accelerate Complex AI Agent Development with Percival

This notebook will illustrate how to use **Percival** to debug Agents!

We tested this out with an Agent that uses the Weaviate Query and Transformation Agents as tools! This Agent is built using the **Function Calling** agentic architecture. **Function Calling** describes an iterative loop where, at each stage, the Agent decides to either call one of the tools or respond to the user. 

The Function Calling Agent we define is given a very open-ended task -- *transform a collection of blog posts that are stored in Weaviate with useful insights and structure*. 

This is a massive paradigm shift in how software is developed. Rather than requiring human developers to painstakingly define every detail of a computational workflow, Agents are able to figure out the workflow to complete tasks for themselves!

However, the problem is that evaluating and controlling these open-ended Agents is much more difficult than traditional software, or machine learning systems. As we will see, this particular Agent utilizes 12 steps of Function Calls before ultimately returning a response to the user. Manually debugging this single trace, let alone several of them, is a daunting and time consuming task!

Enter **Percival**! 

**Percival** is an Agent for inspecting and debugging Agents from Patronus AI!

With the `@patronus.traced()` decorator, all of the LLM inferences and tool calls utilized by our Agent is added to Patronus. From there we can then use Percival to inspect the trace by clicking `"Analyze with Percival"`.

### Tool Output Misinterpretation

In this first example, Percival detects that the Function Calling Agent is incorrectly interpreting the result of a tool. This informs us how to fix the response model in our user defined tools to make it easier for the Function Calling to understand.

![Sunset over mountains](./images/percival-1.png)

### Goal Deviation

In the second example, Percival detects that the Agent has lost sight of it's original goal while processing a long trajectory of Function Calls. This is a common problem when dealing with Long Contexts and LLMs. Similarly to the interpretation of tool outputs, we can put a quick bandaid on this by tweaking the response models of our tools, limiting the amount of context sent back to the Function Calling Agent. This can also inform us how to build alternate Agent Architectures to Function Calling that better handle long contexts.

![Sunset over mountains](./images/percival-2.png)

Check out the [docs](https://docs.patronus.ai/docs/percival/percival) to learn more about getting started with Percival!

In [10]:
import os
import tiktoken
import time

import weaviate
import weaviate.collections.classes.config as wvcc
from dotenv import load_dotenv
from weaviate.classes.init import AdditionalConfig, Timeout

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
)

load_dotenv()

local_blogs = []

# The Weaviate Blogs dataset can be found in recipes within `integrations/llm-agent-frameworks/data`
# You can also get it from `github.com/weaviate-io/blog``
main_folder_path = "./blog/"

for i, folder_name in enumerate(os.listdir(main_folder_path)):
    subfolder_path = os.path.join(main_folder_path, folder_name)
    if os.path.isdir(subfolder_path):
        index_file_path = os.path.join(subfolder_path, "index.mdx")
        if os.path.isfile(index_file_path):
            with open(index_file_path, "r", encoding="utf-8") as file:
                content = file.read()
                local_blogs.append(
                    {
                        "content": content,
                    }
                )

if weaviate_client.collections.exists("Blogs"):
    weaviate_client.collections.delete("Blogs")
blogs = weaviate_client.collections.create(
    name="Blogs",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
    ],
)

def chunk_text(text, max_tokens=300):
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)
    chunks = []
    
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)
    
    return chunks

chunked_blogs = []
for blog in local_blogs:
    chunks = chunk_text(blog["content"])
    for chunk in chunks:
        chunked_blogs.append({
            "content": chunk
        })

start_time = time.time()
with weaviate_client.batch.dynamic() as batch:
    for blog_chunk in chunked_blogs:
        batch.add_object(
            collection="Blogs",
            properties={
                "content": blog_chunk["content"],
            }
        )
end_time = time.time()
upload_time = end_time - start_time

print(f"Successfully imported {len(chunked_blogs)} blog chunks into Weaviate.")
print(f"Upload time: {upload_time:.2f} seconds")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/collections/classes/config.py:1950: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_17038/3726094897.py:67: ResourceWarning: unclosed <ssl.SSLSocket fd=77, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.120', 65497), raddr=('35.201.124.182', 443)>
  with weaviate_client.batch.dynam

Successfully imported 1463 blog chunks into Weaviate.
Upload time: 8.71 seconds


In [ ]:
import patronus
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.instrumentation.threading import ThreadingInstrumentor
from opentelemetry.instrumentation.asyncio import AsyncioInstrumentor
from datetime import datetime
from smolagents import LiteLLMModel, ToolCallingAgent, tool
import weaviate
from weaviate.agents.query import QueryAgent
from weaviate.agents.transformation import TransformationAgent
from weaviate.agents.classes import Operations
from weaviate.collections.classes.config import DataType
import os
from dotenv import load_dotenv
import time

load_dotenv()

patronus.init(integrations=[SmolagentsInstrumentor(), ThreadingInstrumentor()])
@tool
def inspect_random_objects(num_samples: int) -> str:
    """
    Retrieves a random sample of objects from the Weaviate Blogs collection.
    
    Args:
        num_samples: Number of random objects to retrieve. Will be capped at 1000.
    
    Returns:
        A string containing all properties of the sampled objects.
    """
    import random
    
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
    )
    
    collection = weaviate_client.collections.get("Blogs")
    
    result = []
    
    # Sample from the first 1000 objects
    max_objects = 1000
    sample_size = min(num_samples, max_objects)
    random_indices = sorted(random.sample(range(max_objects), sample_size))
    
    # Fetch objects at those indices
    random_sample = []
    current_index = 0
    for i, obj in enumerate(collection.iterator(limit=max_objects)):
        if current_index < len(random_indices) and i == random_indices[current_index]:
            random_sample.append(obj)
            current_index += 1
        if current_index >= len(random_indices):
            break
    
    # Format the results with all properties
    for i, obj in enumerate(random_sample):
        result.append(f"Object {i+1}:")
        for prop_name, prop_value in obj.properties.items():
            result.append(f"  {prop_name}: {prop_value}")
        result.append("")  # Empty line between objects
    
    weaviate_client.close()
    return "\n".join(result) if result else "No objects found in the collection."

@tool
def ask_weaviate_agent(query: str) -> str:
    """
    Returns answers to questions from knowledge stored in a database.

    Args:
        query: the query to ask the database.
    """
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
    )
    print(weaviate_client.is_ready())

    qa = QueryAgent(
        client=weaviate_client, 
        collections=["Blogs"],
    )
    response = qa.run(query)
    weaviate_client.close()
    
    return response

@tool
def transform_weaviate_data(instruction: str, operation_type: str, property_name: str, view_properties: list[str], data_type: str = "BOOL") -> str:
    """
    Starts an asynchronous transformation job that updates or appends properties to the database collection.
    Returns a workflow_id that can be used to check the status of the transformation.

    Args:
        instruction: Instructions for the transformation to perform.
        operation_type: Type of operation to perform - "append" or "update".
        property_name: Name of the property to update or append to.
        view_properties: List of properties to view during transformation. If None, defaults to ["content"].
        data_type: Type of data for the property - "TEXT", "NUMBER", "INT", or "BOOL". Default is "BOOL".
    """
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
    )
    
    # Map string data type to DataType enum
    data_type_map = {
        "TEXT": DataType.TEXT,
        "NUMBER": DataType.NUMBER,
        "INT": DataType.INT,
        "BOOL": DataType.BOOL
    }
    
    if data_type not in data_type_map:
        return f"Invalid data type: {data_type}. Must be one of {', '.join(data_type_map.keys())}."
    
    if operation_type.lower() == "append":
        operation = Operations.append_property(
            property_name=property_name,
            data_type=data_type_map[data_type],
            view_properties=view_properties,
            instruction=instruction,
        )
    elif operation_type.lower() == "update":
        operation = Operations.update_property(
            property_name=property_name,
            view_properties=view_properties,
            instruction=instruction,
        )
    else:
        return f"Invalid operation type: {operation_type}. Must be 'append' or 'update'."
    
    agent = TransformationAgent(
        client=weaviate_client,
        collection="Blogs",
        operations=[operation],
    )
    
    response = agent.update_all()
    weaviate_client.close()
    
    return f"Transformation started. Workflow ID: {response.workflow_id}. Use check_transformation_status tool to monitor progress."

@tool
def check_transformation_status(workflow_id: str) -> str:
    """
    Checks the status of an asynchronous transformation job.
    Please note the Transformation Agent typically takes a few minutes to run.

    Args:
        workflow_id: The ID of the workflow to check, obtained from transform_weaviate_data.
    """
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
    )
    
    agent = TransformationAgent(
        client=weaviate_client,
        collection="Blogs",
        operations=[]
    )
    
    status = agent.get_status(workflow_id=workflow_id)
    weaviate_client.close()
    
    return f"Transformation status: {status}"

@tool
def check_blogs_schema() -> str:
    """
    Retrieves the details of the Blogs collection stored in Weaviate.
    
    Returns:
        A string representation of the Blogs schema in the database.
    """
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
    )
    
    response = weaviate_client.collections.list_all(simple=True)["Blogs"]
    weaviate_client.close()
    
    return str(response)

@tool
def wait_for_seconds(seconds: int) -> str:
    """
    Waits for the specified number of seconds before continuing.
    Useful when waiting for asynchronous operations to complete.

    IMPORTANT!! Please note, a Transformation Agent operation on 1,000 documents typically takes about 3 minutes.
    
    After initiating a transformation with transform_weaviate_data, use this tool to wait for at least 
    60 seconds before calling check_transformation_status. Increase the waiting time if the number of 
    documents being transformed is greater than 1000. Only check the transformation status periodically, 
    not continuously.

    Args:
        seconds: Number of seconds to wait.
    """
    time.sleep(seconds)
    return f"Waited for {seconds} seconds."

def create_agent(model_id):
  model = LiteLLMModel(model_id, temperature=0., top_p=1.)
  qa_agent = ToolCallingAgent(
    tools=[ask_weaviate_agent, transform_weaviate_data, check_transformation_status, check_blogs_schema, wait_for_seconds],
    model=model,
    max_steps=20,
    name="weaviate_agent",
    description="""
You are connected to a search engine that lets you search for information contained in the Weaviate Blogs.
You can also transform data in the blogs collection database by adding or updating properties.

Note that transformations run asynchronously. When you call transform_weaviate_data, you'll receive a workflow_id.
Use the check_transformation_status tool with this workflow_id to monitor progress.
If a transformation is still running, you can use the wait_for_seconds tool to pause before checking again.
Please note the Transformation Agent typically takes a few *minutes* to run.
You can also check the Blogs schema in the database using the check_weaviate_schemas tool. This is useful for planning Transformation Agent operations.
Before calling `transform_weaviate_data`, check if the target property already exists in the collection using `check_weaviate_schemas`. If the property exists, either skip the transformation or update the instruction to modify the existing property instead of creating a new one.

Explore the blog content and develop a strategy to enhance it. You have complete freedom to decide what metadata would be most valuable to add. Consider:

1. What themes, topics, or patterns exist in the content?
2. What metadata would make this content more searchable and useful?
3. How might you categorize or tag this content to improve navigation?

Your task:
- First, explore the content to understand what you're working with
- Design and implement a metadata schema that adds value (you decide what properties to add)
- Transform the content by adding at least 3 new properties of your choice
- Include a variety of property types in your schema:
  * TEXT properties for descriptive content like summaries, categories, or tags
  * INT properties for numerical data like reading time, publication year, or complexity scores
  * BOOL properties for binary classifications like "is_tutorial", "has_code_examples", or "is_beginner_friendly"
- After your transformations, analyze the enhanced content and create a report that:
  * Explains your metadata strategy and why you chose those properties
  * Provides examples of how your enhancements improve content discovery
  * Suggests ways users could leverage these new properties
  * Recommends additional improvements for future iterations

You have complete creative freedom - there's no single "right answer." The goal is to demonstrate your ability to analyze content, identify valuable metadata opportunities, and implement them effectively. Don't overfit on TEXT properties - a balanced schema with different property types will provide more versatile search and filtering capabilities.
Before summarizing, ALWAYS use the ask_weaviate_agent tool to analyze blog content and identify key features and benefits.
"""
  )
  return qa_agent


@patronus.traced()
def main():
    agent = create_agent("openai/gpt-4o")
    
    complex_query = """
You're a content strategist tasked with organizing a large collection of technical blog posts. Your goal is to extract structure and insights that makes this content more discoverable and useful.
    """
    
    agent.run(complex_query)

main()

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_17038/825335243.py:18: UserWarning: The Patronus SDK has already been initialized. Duplicate initialization attempts are ignored.
  patronus.init(integrations=[SmolagentsInstrumentor(), ThreadingInstrumentor()])


╭─────────────────────────────────────────── New run - weaviate_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a content strategist tasked with organizing a large collection of technical blog posts. Your goal is to  │
│ extract structure and insights that makes this content more discoverable and useful.                            │
│                                                                                                                 │
│ Explore the blog content and develop a strategy to enhance it. You have complete freedom to decide what         │
│ metadata would be most valuable to add. Consider:                                                               │
│                                                                                                                 │
│ 1. What themes, topics, or patterns exist in the content?                                                       │
│ 2. What metadata would make this content more searchable and useful?                                            │
│ 3. How might you categorize or tag this content to improve navigation?                                          │
│                                                                                                                 │
│ Your task:                                                                                                      │
│ - First, explore the content to understand what you're working with                                             │
│ - Design and implement a metadata schema that adds value (you decide what properties to add)                    │
│ - Transform the content by adding at least 3 new properties of your choice                                      │
│ - Include a variety of property types in your schema:                                                           │
│   * TEXT properties for descriptive content like summaries, categories, or tags                                 │
│   * INT properties for numerical data like reading time, publication year, or complexity scores                 │
│   * BOOL properties for binary classifications like "is_tutorial", "has_code_examples", or                      │
│ "is_beginner_friendly"                                                                                          │
│ - After your transformations, analyze the enhanced content and create a report that:                            │
│   * Explains your metadata strategy and why you chose those properties                                          │
│   * Provides examples of how your enhancements improve content discovery                                        │
│   * Suggests ways users could leverage these new properties                                                     │
│   * Recommends additional improvements for future iterations                                                    │
│                                                                                                                 │
│ You have complete creative freedom - there's no single "right answer." The goal is to demonstrate your ability  │
│ to analyze content, identify valuable metadata opportunities, and implement them effectively. Don't overfit on  │
│ TEXT properties - a balanced schema with different property types will provide more versatile search and        │
│ filtering capabilities.                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'check_blogs_schema' with arguments: {}                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: _CollectionConfigSimple(name='Blogs', description=None, generative_config=None, 
properties=|_Property(name='content', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, 
index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>,
vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), 
vectorizer='text2vec-weaviate', vectorizer_configs=None)], references=|], reranker_config=None, 
vectorizer_config=_VectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_WEAVIATE: 'text2vec-weaviate'>, 
model={'baseUrl': 'https://api.embedding.weaviate.io', 'model': 'Snowflake/snowflake-arctic-embed-l-v2.0', 
'truncate': 'right'}, vectorize_collection_name=True), vectorizer=<Vectorizers.TEXT2VEC_WEAVIATE: 
'text2vec-weaviate'>, vector_config=None)

...

[Step 11: Duration 2.61 seconds| Input tokens: 36,612 | Output tokens: 360]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 12 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '### Metadata Strategy and Implementation\n\n**1.       │
│ Themes, Topics, and Patterns:**\nThe blog content often includes deep research, authoritative references,       │
│ structured insights, and visual elements like images or graphs. Common topics include AI-focused content,       │
│ technical integrations, and practical applications in various industries.\n\n**2. Metadata Schema Design:**\nTo │
│ enhance discoverability and usability, I implemented the following metadata properties:\n- **Metadata (TEXT):** │
│ Provides a summary or key themes of the blog post, making it easier to understand the content at a glance.\n-   │
│ **Reading Time (INT):** Estimates the time required to read the blog post, helping users manage their reading   │
│ schedule.\n- **Has Code Examples (BOOL):** Indicates whether the blog post contains code examples, useful for   │
│ developers looking for practical coding insights.\n\n**3. Transformation Process:**\n- Added a `metadata`       │
│ property to provide a concise summary of each blog post.\n- Calculated and appended `reading_time` based on     │
│ word count.\n- Added a `has_code_examples` property to identify posts with code snippets.\n\n### Analysis and   │
│ Report\n\n**Metadata Strategy Explanation:**\n- **Metadata:** Helps users quickly identify the relevance of a   │
│ blog post to their interests.\n- **Reading Time:** Assists in time management and planning for readers.\n-      │
│ **Has Code Examples:** Facilitates quick identification of posts with practical coding content.\n\n**Examples   │
│ of Enhanced Content Discovery:**\n- Users can filter blog posts by reading time to find content that fits their │
│ available time.\n- Developers can search for posts with code examples to find practical guides and              │
│ tutorials.\n\n**User Leverage Suggestions:**\n- Use metadata summaries to quickly assess the relevance of       │
│ content.\n- Filter by reading time to prioritize shorter or longer reads based on availability.\n- Search for   │
│ posts with code examples to enhance learning with practical applications.\n\n**Recommendations for Future       │
│ Improvements:**\n- Add a `publication_year` property to track the currency of the content.\n- Implement a       │
│ `complexity_score` to help users find content that matches their expertise level.\n- Consider adding a          │
│ `category` property to further refine search and filtering capabilities.\n\nThis balanced schema with TEXT,     │
│ INT, and BOOL properties provides versatile search and filtering capabilities, enhancing the overall user       │
│ experience.'}                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: ### Metadata Strategy and Implementation

**1. Themes, Topics, and Patterns:**
The blog content often includes deep research, authoritative references, structured insights, and visual elements 
like images or graphs. Common topics include AI-focused content, technical integrations, and practical applications
in various industries.

**2. Metadata Schema Design:**
To enhance discoverability and usability, I implemented the following metadata properties:
- **Metadata (TEXT):** Provides a summary or key themes of the blog post, making it easier to understand the 
content at a glance.
- **Reading Time (INT):** Estimates the time required to read the blog post, helping users manage their reading 
schedule.
- **Has Code Examples (BOOL):** Indicates whether the blog post contains code examples, useful for developers 
looking for practical coding insights.

**3. Transformation Process:**
- Added a `metadata` property to provide a concise summary of each blog post.
- Calculated and appended `reading_time` based on word count.
- Added a `has_code_examples` property to identify posts with code snippets.

### Analysis and Report

**Metadata Strategy Explanation:**
- **Metadata:** Helps users quickly identify the relevance of a blog post to their interests.
- **Reading Time:** Assists in time management and planning for readers.
- **Has Code Examples:** Facilitates quick identification of posts with practical coding content.

**Examples of Enhanced Content Discovery:**
- Users can filter blog posts by reading time to find content that fits their available time.
- Developers can search for posts with code examples to find practical guides and tutorials.

**User Leverage Suggestions:**
- Use metadata summaries to quickly assess the relevance of content.
- Filter by reading time to prioritize shorter or longer reads based on availability.
- Search for posts with code examples to enhance learning with practical applications.

**Recommendations for Future Improvements:**
- Add a `publication_year` property to track the currency of the content.
- Implement a `complexity_score` to help users find content that matches their expertise level.
- Consider adding a `category` property to further refine search and filtering capabilities.

This balanced schema with TEXT, INT, and BOOL properties provides versatile search and filtering capabilities, 
enhancing the overall user experience.

[Step 12: Duration 8.07 seconds| Input tokens: 40,977 | Output tokens: 854]